<a href="https://colab.research.google.com/github/JorWo/Student-Performance-Classification/blob/main/Student_Performance_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###
### Imports
###

import pandas as pd
import numpy as np
import matplotlib as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential
from keras.layers import Dense, Input

# Describe columns (dataset info: https://archive.ics.uci.edu/ml/datasets/Higher+Education+Students+Performance+Evaluation+Dataset)
col_names = ['studentID', 'age', 'sex', 'graduatedHighSchoolType', 'scholarshipType', 'additionalWork', 'regularArtisticOrSportsActivity', 'hasPartner', 'totalSalary', 'transportationToUniversity',
             'housingSituation', 'mothersEducation', 'fathersEducation', 'numSiblings', 'parentalStatus', 'mothersOccupation', 'fathersOccupation', 'weeklyStudyHrs', 'fictionReadingFreq',
             'nonfictionReadingFreq', 'seminarConferenceAttendance', 'projectActivityImpact', 'classAttendance', 'midtermExamPrepWhom', 'midtermExamPrepWhen', 'classNoteTakingFreq',
             'classListeningFreq', 'classDiscussionInterest', 'flipClassroomOpinion', 'lastSemCumGPA', 'expectedGradCumGPA', 'courseID', 'outputGrade']

# Load dataset
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00623/DATA.csv', sep=';', names=col_names).drop(index=0) # drop first row (removes original column names)

# Seperate features and classes
X = df.drop(columns=['studentID', 'outputGrade'])
y = df['outputGrade']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
###
### Feature Selection With Decision Tree
###

# Train Decision Tree
dt = DecisionTreeClassifier(min_samples_split=3, random_state=42)
dt.fit(X_train, y_train)

# Visualize feature importance in DataFrame
feature_importances = pd.DataFrame(dt.feature_importances_, index=X_train.columns, columns=['Importance'])
feature_importances.sort_values(by='Importance', ascending=False, inplace=True)

# Drop features with importance == 0
X_train = X_train.drop(columns=['midtermExamPrepWhen', 'hasPartner', 'transportationToUniversity', 'graduatedHighSchoolType', 'sex'])
X_test = X_test.drop(columns=['midtermExamPrepWhen', 'hasPartner', 'transportationToUniversity', 'graduatedHighSchoolType', 'sex'])


In [ ]:
###
### Technique #1: K-Nearest Neighbors
###

knn = KNeighborsClassifier(n_neighbors=3, algorithm='ball_tree')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))

Accuracy: 0.3783783783783784


In [ ]:
###
### Technique #2: Random Forest
###

rf = RandomForestClassifier(n_estimators=300, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))

Accuracy: 0.32432432432432434


In [ ]:
###
### Technique #3: Neural Network
###

# Create neural network
nnet = Sequential()

# Add input layer
nnet.add(Input(shape=(X_train.shape[1],)))

# Add hidden layers
nnet.add(Dense(100, activation='relu'))
nnet.add(Dense(50, activation='relu'))
nnet.add(Dense(100, activation='relu'))

# Add output layer
nnet.add(Dense(len(np.unique(y_train)), activation='softmax'))

# Compile
nnet.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Set to int as Keras does not recognize them
X_train = np.asarray(X_train).astype(int)
y_train = np.asarray(y_train).astype(int)
X_test = np.asarray(X_test).astype(int)
y_test = np.asarray(y_test).astype(int)

# Fit
nnet.fit(X_train, y_train, epochs=100, batch_size=1)

# Display test accuracy
print('\n')
score, accuracy = nnet.evaluate(X_test, y_test)
print('Accuracy:', accuracy)

Epoch 1/100
108/108 [==============================] - 2s 3ms/step - loss: 2.0845 - accuracy: 0.1759
Epoch 2/100
108/108 [==============================] - 0s 3ms/step - loss: 1.9543 - accuracy: 0.1852
Epoch 3/100
108/108 [==============================] - 0s 3ms/step - loss: 1.8069 - accuracy: 0.2407
Epoch 4/100
108/108 [==============================] - 0s 3ms/step - loss: 1.7629 - accuracy: 0.2407
Epoch 5/100
108/108 [==============================] - 0s 3ms/step - loss: 1.7018 - accuracy: 0.2593
Epoch 6/100
108/108 [==============================] - 0s 3ms/step - loss: 1.6366 - accuracy: 0.3519
Epoch 7/100
108/108 [==============================] - 0s 3ms/step - loss: 1.5858 - accuracy: 0.3796
Epoch 8/100
108/108 [==============================] - 0s 3ms/step - loss: 1.5424 - accuracy: 0.3426
Epoch 9/100
108/108 [==============================] - 0s 3ms/step - loss: 1.4890 - accuracy: 0.3519
Epoch 10/100
108/108 [==============================] - 0s 3ms/step - loss: 1.4335 - accura